# setup

In [ ]:
PATH_ARQUIVOS = r"data"
LANG = "por" # por eng
persist_directory = "chroma/chroma_db"  # Diretório onde o banco de dados será salvo

## torch

In [ ]:
import torch

# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

## docling

In [ ]:
from docling.datamodel.base_models import InputFormat
from docling_core.types.doc import ImageRefMode
from docling.document_converter import DocumentConverter, PdfFormatOption
from docling.datamodel.pipeline_options import PdfPipelineOptions, TableFormerMode, EasyOcrOptions, TesseractOcrOptions, OcrMacOptions
from docling.datamodel.settings import settings

IMAGE_RESOLUTION_SCALE = 2.0

# Define pipeline options for PDF processing
pipeline_options = PdfPipelineOptions(
    do_table_structure=True,  # Enable table structure detection
    do_ocr=True,  # Enable OCR
    # full page ocr and language selection
    #ocr_options=EasyOcrOptions(force_full_page_ocr=True, lang=["en"]),  # Use EasyOCR for OCR
    ocr_options=TesseractOcrOptions(force_full_page_ocr=True, lang=[LANG]),  # Uncomment to use Tesseract for OCR
    #ocr_options = OcrMacOptions(force_full_page_ocr=True, lang=['en-US']),
    table_structure_options=dict(
        do_cell_matching=False,  # Use text cells predicted from table structure model
        mode=TableFormerMode.ACCURATE  # Use more accurate TableFormer model
    ),
    generate_page_images=True,  # Enable page image generation
    generate_picture_images=True,  # Enable picture image generation
    images_scale=IMAGE_RESOLUTION_SCALE, # Set image resolution scale (scale=1 corresponds to a standard 72 DPI image)
)

# Initialize the DocumentConverter with the specified pipeline options
doc_converter_global = DocumentConverter(
    format_options={
        InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)
    }
)

## Testes

### docling test

In [ ]:
from pathlib import Path
from PyPDF2 import PdfReader, PdfWriter

# Read the PDF file
reader = PdfReader(r"data\pdfs\WEF_The_Global_Cooperation_Barometer_2024.pdf")
writer = PdfWriter()

OUTPUT = Path(r"data\pdfs\page_14.pdf")

# Ensure the requested page exists in the document
if len(reader.pages) >= 14:
    # Add page 14 (index 13 since it's 0-based)
    writer.add_page(reader.pages[13])
else:
    print("The PDF does not contain 14 pages.")

# Save the extracted page to a new PDF file
with OUTPUT.open('wb') as output_pdf:
    writer.write(output_pdf)

print(f"Page 14 has been saved to {OUTPUT}")

In [ ]:
result = doc_converter_global.convert(OUTPUT)
display(result.document.export_to_markdown(image_mode=ImageRefMode.EMBEDDED))

In [ ]:
if OUTPUT.exists():
    OUTPUT.unlink()
    print(f"Arquivo {OUTPUT} excluído com sucesso.")
else:
    print(f"Arquivo {OUTPUT} não existe.")

### html

In [ ]:
file = Path(r"data\html\python - How to run DeepSeek model locally - Stack Overflow.html")

result = doc_converter_global.convert(file)
display(result.document.export_to_markdown(image_mode=ImageRefMode.EMBEDDED))

### diversos

In [ ]:
file = Path(r"data\docs\word.docx")

result = doc_converter_global.convert(file)
display(result.document.export_to_markdown(image_mode=ImageRefMode.EMBEDDED))

In [ ]:
file = Path(r"data\docs\xlsx.xlsx")

result = doc_converter_global.convert(file)
display(result.document.export_to_markdown(image_mode=ImageRefMode.EMBEDDED))

### pytesseract test

In [ ]:
import pytesseract
from PIL import Image

#pytesseract.pytesseract.tesseract_cmd = r"E:\programas\ia\Tesseract-OCR\tesseract.exe"

image_path = r"data\imagens\Captura de tela 2025-03-13 085540.png"
image = Image.open(image_path)

extracted_text = pytesseract.image_to_string(image, lang=LANG)
print(extracted_text)

### Chunking

#### direct chunking

In [ ]:
result = doc_converter_global.convert(Path(r"data\pdfs\monopoly.pdf"))
display(result.document.export_to_markdown(image_mode=ImageRefMode.EMBEDDED))

In [ ]:
from docling.document_converter import DocumentConverter
from docling.chunking import HybridChunker

#conv_res = DocumentConverter().convert(FIRST_10_PAGES)
#doc = conv_res.document

chunker = HybridChunker(tokenizer="BAAI/bge-small-en-v1.5")  # set tokenizer as needed
chunk_iter = chunker.chunk(result.document)

# Convert the iterator to a list to count the chunks
chunks = list(chunk_iter)
num_chunks = len(chunks)

# Print the number of chunks
print(f"The document has been divided into {num_chunks} chunks.")

In [ ]:
print(chunks[1].text)

#### chroma chunking

obs: precisa do ollama executando `ollama serve`

In [ ]:
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

In [ ]:
# display(result.document.export_to_markdown())
from langchain_core.documents import Document

local_path = r"data\pdfs\monopoly.pdf"
result = doc_converter_global.convert(Path(local_path))
documento = Document(page_content=result.document.export_to_markdown(image_mode=ImageRefMode.EMBEDDED), metadata={"source": local_path})

print(documento)

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=7500, chunk_overlap=100)
chunks = text_splitter.split_documents([documento])

In [ ]:
print(f"num chunks: {len(chunks)}")
print(chunks[0])

# Análise documentos

In [ ]:
import os

def separar_arquivos(diretorio):
    """
    Varre um diretório e suas subpastas, separando arquivos de imagem de outros tipos de arquivo.

    Args:
        diretorio (str): O caminho do diretório a ser varrido.

    Returns:
        tuple: Uma tupla contendo duas listas: imagens e documentos.
    """

    imagens = []
    documentos = []

    extensoes_imagens = ['.jpg', '.jpeg', '.png', '.gif', '.bmp']  # Adicione outras extensões se necessário

    for raiz, subpastas, arquivos in os.walk(diretorio):
        for arquivo in arquivos:
            caminho_arquivo = os.path.join(raiz, arquivo)
            nome_arquivo, extensao = os.path.splitext(arquivo)
            extensao = extensao.lower()

            if extensao in extensoes_imagens:
                imagens.append(caminho_arquivo) #adiciona o caminho completo
            else:
                documentos.append(caminho_arquivo) #adiciona o caminho completo

    return imagens, documentos

diretorio = r'data'
imagens, documentos = separar_arquivos(diretorio)

# print("Imagens:")
# for imagem in imagens: print(imagem)

# print("\nDocumentos:")
# for documento in documentos: print(documento)

# Chromadb

obs: precisa do ollama executando `ollama serve`

## chroma batch

In [ ]:
import hashlib
import os
import chromadb
from chromadb.config import Settings
from langchain.docstore.document import Document
#from langchain_community.embeddings import OllamaEmbeddings
from langchain_ollama import OllamaEmbeddings 
from pathlib import Path

def generate_id(document, page_index):
    """Gera um ID único baseado no nome do arquivo e no índice da página."""
    source = document.metadata['source']
    filename = os.path.basename(source)
    base_id = hashlib.sha256(filename.encode()).hexdigest()
    return f"{base_id}_{page_index}"

# no batch

# def check_and_add_document(collection, document, page_index, embedding_model):
#     """Verifica se um documento já existe na coleção e o adiciona se não existir."""
#     document_id = generate_id(document, page_index)

#     # Verifica se o ID já existe na coleção
#     results = collection.get(ids=[document_id])
#     if results['ids'] and document_id in results['ids']:
#         print(f"Documento com ID {document_id} já existe na coleção.")
#         return

#     # Adiciona o documento à coleção
#     embedding = embedding_model.embed_documents([document.page_content])[0]
#     collection.add(documents=[document.page_content], ids=[document_id], embeddings=[embedding], metadatas=[document.metadata])
#     print(f"Documento com ID {document_id} adicionado à coleção.")

# # exemplo inline de chunks
# # chunks = [
# #     Document(page_content="This is a document about pineapple", metadata={"source": "file1.txt"}),
# #     Document(page_content="This is a document about oranges", metadata={"source": "file2.txt"}),
# #     Document(page_content="Another document about pineapple", metadata={"source": "file1.txt"}),
# #     Document(page_content="New document about oranges", metadata={"source": "file2.txt"}),
# # ]

# embedding_model = OllamaEmbeddings(model="nomic-embed-text")
# # Inicializa o cliente ChromaDB
# chroma_client = chromadb.PersistentClient(path=persist_directory, settings=Settings(allow_reset=False))

# collection_name = "local-rag"

# def chroma_indexing(chunks, collection_name = "local-rag"):
#     # Obtém ou cria a coleção
#     collection = chroma_client.get_or_create_collection(name=collection_name)
#     print("Coleção carregada.")

#     # Itera sobre os chunks e os adiciona à coleção, verificando se já existem
#     for i, chunk in enumerate(chunks):
#         check_and_add_document(collection, chunk, i, embedding_model)

#     # Exemplo para verificar o ID do primeiro chunk
#     if chunks:
#         first_chunk_id = generate_id(chunks[0], 0)
#         print(f"ID do primeiro chunk: {first_chunk_id}")

#     # # Exemplo de query para verificar se os dados foram adicionados corretamente
#     # query_embedding = embedding_model.embed_query("fruit information")
#     # query_results = collection.query(query_embeddings=[query_embedding], n_results=2)
#     # print("\nResults of query:")
#     # for result in query_results['documents'][0]:
#     #     print(result)
#     return collection

# batch

embedding_model = OllamaEmbeddings(model="nomic-embed-text")
chroma_client = chromadb.PersistentClient(path=persist_directory, settings=Settings(allow_reset=False))
collection_name = "local-rag"

def chroma_indexing_batch(chunks, collection_name="local-rag", embedding_model=None, chroma_client=None):
    """Indexa chunks em lote no ChromaDB."""
    if not chunks:
        return

    collection = chroma_client.get_or_create_collection(name=collection_name)
    print("Coleção carregada.")

    documents_to_add = []
    ids_to_add = []
    embeddings_to_add = []
    metadatas_to_add = []

    for i, chunk in enumerate(chunks):
        document_id = generate_id(chunk, i)
        results = collection.get(ids=[document_id])

        if results['ids'] and document_id in results['ids']:
            print(f"Documento com ID {document_id} já existe na coleção.")
            continue

        embedding = embedding_model.embed_documents([chunk.page_content])[0]

        documents_to_add.append(chunk.page_content)
        ids_to_add.append(document_id)
        embeddings_to_add.append(embedding)
        metadatas_to_add.append(chunk.metadata)

    if documents_to_add:
        collection.add(documents=documents_to_add, ids=ids_to_add, embeddings=embeddings_to_add, metadatas=metadatas_to_add)
        print(f"Adicionados {len(documents_to_add)} documentos em lote.")

    if chunks:
        first_chunk_id = generate_id(chunks[0], 0)
        print(f"ID do primeiro chunk: {first_chunk_id}")

    return collection



## Document chunks

In [ ]:
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

# display(result.document.export_to_markdown())
from langchain_core.documents import Document

# img
import pytesseract
from PIL import Image
#pytesseract.pytesseract.tesseract_cmd = r"E:\programas\ia\Tesseract-OCR\tesseract.exe"
#end img

text_splitter = RecursiveCharacterTextSplitter(chunk_size=7500, chunk_overlap=100)

#local_path = r"data\pdfs\monopoly.pdf"
def get_chunks_doc(local_path):
    result = doc_converter_global.convert(Path(local_path))
    documento = Document(page_content=result.document.export_to_markdown(image_mode=ImageRefMode.EMBEDDED), metadata={"source": local_path})
    chunks = text_splitter.split_documents([documento])
    return chunks


def get_chunks_image(local_path):
    image = Image.open(local_path)
    extracted_text = pytesseract.image_to_string(image, lang=LANG)

    documento = Document(page_content=extracted_text, metadata={"source": local_path})
    chunks = text_splitter.split_documents([documento])
    return chunks

## Chroma indexing

In [10]:
# collection_name = "local-rag"

# for imagem in imagens:
#     chunks = get_chunks_image(imagem)
#     if (chunks is None or len(chunks) <= 0): continue
#     chroma_indexing(chunks, collection_name)

# for documento in documentos:
#     chunks = get_chunks_doc(documento)
#     if (chunks is None or len(chunks) <= 0): continue
#     chroma_indexing(chunks, collection_name)

# print("Documentos indexados")

for imagem in imagens:
    chunks = get_chunks_image(imagem)
    if chunks:
        chroma_indexing_batch(chunks, collection_name, embedding_model, chroma_client)

for documento in documentos:
    chunks = get_chunks_doc(documento)
    if chunks:
        chroma_indexing_batch(chunks, collection_name, embedding_model, chroma_client)

print("Documentos indexados")

# chunks_list = []

# for imagem in imagens:
#     chunks = get_chunks_image(imagem)
#     if chunks: chunks_list.append(chunks)

# for documento in documentos:
#     chunks = get_chunks_doc(documento)
#     if chunks: chunks_list.append(chunks)

# chroma_indexing_batch(chunks_list, collection_name, embedding_model, chroma_client)

# print("Documentos indexados")


Coleção carregada.
Adicionados 348 documentos em lote.
ID do primeiro chunk: 7c0100cbcab341da2d7e3063ea01534e24cd669c505f7845f3ccc8bd95ddba50_0
Coleção carregada.
Adicionados 82 documentos em lote.
ID do primeiro chunk: 35ee8ca3231e1a9d9ea39b17b42b8c0fc604929073ef313ce42096db30ea7847_0
Coleção carregada.
Adicionados 266 documentos em lote.
ID do primeiro chunk: 6067fc7624ab65a60d140267e037bb518f47f035d79cba85a3bfd421e6007c3a_0
Coleção carregada.
Adicionados 10 documentos em lote.
ID do primeiro chunk: a8f083315429e2f971a0292de1197fef3a7059aa7ef6a1698b1c1d2c0775341e_0
Coleção carregada.
Adicionados 760 documentos em lote.
ID do primeiro chunk: 27c24834bbaf23c523d8d555b4026d3e51710cc16d86a1d1eb55092a793fcbaa_0
Coleção carregada.
Adicionados 1401 documentos em lote.
ID do primeiro chunk: 1cb071d312447aff4693e8b3dffd19573f21df379dfdb79defcca0fd40b25792_0
Documentos indexados


In [11]:
collection = chroma_client.get_or_create_collection(name=collection_name)
results = collection.get(limit=5)
display(results)

{'ids': ['ff355d2b2b6da4cf0f4fff0d3a48f8d7bf920e8e9fd5854d743d94b8d74431cb_0',
  '4de61c5de3846ef479128ce449eca7c0e6bacbb02259525bd3120ed54b454ead_0',
  '4de61c5de3846ef479128ce449eca7c0e6bacbb02259525bd3120ed54b454ead_1',
  '4de61c5de3846ef479128ce449eca7c0e6bacbb02259525bd3120ed54b454ead_2',
  '4de61c5de3846ef479128ce449eca7c0e6bacbb02259525bd3120ed54b454ead_3'],
 'embeddings': None,
 'documents': ['4)\n\ndocli\n\nimpo:\nfrom\nfrom\nfrom\nfrom\nfrom\n\nng\n\nrt time\n\ndocling.datamodel.base models import InputFormat\n\ndocling core.types.doc import ImageRefMode\n\ndocling.document converter import DocumentConverter, PdfFormatOption\n\ndocling.datamodel .pipeline options import PdfPipelineOptions, TableFormerMode, EasyOcrOptions, TesseractOcrOptions, OcrMacOptions\n\ndocling.datamodel .settings import settings\n\nIMAGE. RESOLUTION SCALE = 2.0\n\n* Define pipeline options for PDF processing\n\npipe\n\n)\n\n4 In\ndoc :\n\nQ 92s\n\n\'line options = PdfPipelineOptions(\n\ndo table struct

# Retrieval

In [12]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

In [13]:
# LLM from Ollama
local_model = "llama3.2"
local_model = "deepseek-r1"
llm = ChatOllama(model=local_model)

In [14]:
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""Você é um assistente de modelo de linguagem de IA. Sua tarefa é gerar cinco
    versões diferentes da pergunta do usuário fornecida para recuperar documentos relevantes de
    um banco de dados vetorial. Ao gerar múltiplas perspectivas sobre a pergunta do usuário, seu
    objetivo é ajudar o usuário a superar algumas das limitações da pesquisa de similaridade 
    baseada em distância. Forneça essas perguntas alternativas separadas por quebras de linha.
    Pergunta original: {question}""",
)

In [15]:
from langchain_ollama import OllamaEmbeddings 
from langchain_chroma import Chroma

embedding_model = OllamaEmbeddings(model="nomic-embed-text")

#vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding_model)
vector_db = Chroma(
    client=chroma_client,
    collection_name=collection_name,
    embedding_function=embedding_model
)

retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)

# RAG prompt
template = """Responda à pergunta com base SOMENTE no seguinte contexto:
{context}
Pergunta: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [16]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [26]:
from IPython.display import display, Markdown
display(Markdown(chain.invoke("Como jogar Monopoly ?")))

<think>
Primeiro, lembro que o jogo Monopoly é um jogo de tabuleiro clássico em que os jogadores moveem their tokens nas casas do tabuleiro, obeyindo as regras e tentando ganhar dinheiro comprando e vendendo propriedades. O objetivo geral é acumular mais moedas do que os outros jogadores.

Para entender como jogar Monopoly, preciso familiarizar-se com o componente principal: a fundamentals de ruleset, que contém todas as regras do jogo. Isso inclui como se movimenta, como se manipula a moedas, como funciona o brushing (colisões) e os pagamentos, como a imóvel de paradoxo e as special edições, como os jardins e as ruízas.

Outro passo é aprender sobre o começo do jogo. Geralmente, os jogadores sortearão um space e escolherão um ibile para commencer. O jogador que der届 à esquerda da mesa vai ser o "Primeiro Jogador", e os demais seguem em rota. Em seguida, cada jogador escolhe uma propriedade e deve pagar a entrada correspondente.

Depois de cada jogador ter escolhido sua propriedade inicial, o jogo passa para a etapa de ferias. Neste momento, cada jogador pode optar por manter suas propriedades ou vender-las, tentando ganhar dinheiro. Se um jogador der à esquerda da mesa, ele ganha a "free rent" (gratuita), que geralmente significa que ele pode的选择 uma propriedade livre.

Aqui está onde a complexidade do jogo entra em brilho. Cada propriedade tem um preço inicial, e muitas delas também incluem specialidades, como quartos de bilionária ou parques specially lang, que podem oferecer mais valor ou vantagens ao jogador que as compre.

Outra parte crucial é o brushing, ou colisões. Se um jogador está andando e uma casinha ou specialidade aleatória é sortida, ele deve "bruck" para ela. Depende do tipo de casinha qual a regra varia. Por exemplo, moveda aleatória pode fazer o jogador parar, enquanto a imóvel de paradoxo põe o jogador em火烧.

As special edições, como os de jardins ou ruízas, adicionam mais levels de interação e ganhos ao jogo. por exemplo, um jardim de rose ou uma rua de negativo podem oferecer oportunidades adicionais de ganho ou perda de dinheiro.

Uma parte importante também é o estoque de moedas. Jogadores usam as moedas sorteadas para pagar as despesas e manter suas propriedades em andamento. Se um jogador não tiver suficiente moedas, pode ser atraído para a simplesidade de vender uma propriedade para ganhar dinheiro.

Ainda, o jogo inclui specialidades como os esquema de jogos, que permitem que jogadores "bruck" para espaço se as specialidades forem sortidas. Talvez tenhamos um esquema de jogos de jardins ou imóveis de paradoxo que possam oferecer mais beneficios.

Adiante está a etapa de vencedor. O objetivo é ganhar dinheiro, mas em Monopoly, há também o Prêmio do Global, que é premiado a quem tem o maior patrimônio no final de um período determinado. Além disso, há a possibilidade de um jogador ser "empréstimo" ou "devedor" de uma propriedade, influenciando o fluxo de dinheiro entre os jogadores.

Em resumo, para aprender a jogar Monopoly, é essencial entender as regras básicas, a etapa de começo, como lidar com brushing e specialidades, e como manipular o estoque de moedas para manter um equilíbrio entre ganho de dinheiro e manutenção de propriedades. Com time spent practicando e experimentando diferentes战略s, um jogador pode melhorar suas habilidades e aumentar as chances de vencer.
</think>

**Começando a jogar Monopoly:**

1. **Regras do Jogo:**
   - Revise o fundamentals de ruleset para familiarizar-se com todas as regras do jogo, incluindo movimentos, manutenção de propriedades, brushing (colisões), pagamentos e specialidades como a imóvel de paradoxo.

2. **Começo do Jogo:**
   - Sorteie um espaço inicial para determinar o primeiro jogador.
   - Cada jogador escolhe uma propriedade initial, pagando a entrada correspondente.

3. **Férias e Esquema de Jogos:**
   - Durante as férias, cada jogador pode optar por manter suas propriedades ou vender-las.
   - Se um jogador está à esquerda da mesa, recebe a "free rent" (gratuita) e pode escolher uma propriedade livre.

4. ** lidar com Brushing:**
   - Caso uma casinha ou specialidade aleatória seja sortida, o jogador deve "bruck" para ela.
   - Cada tipo de casinha tem regras específicas; por exemplo, a imóvel de paradoxo põe o jogador em火烧.

5. **Specialidades:**
   - As specialidades como jardins e ruízas adicioneam mais levels de interação e possuem influências adicionais no jogo.
   - Alguns esquemas de jogos permitem que jogadores "bruck" para espaço se as specialidades forem sortidas.

6. **Ganhando Dinheiro:**
   - Jogadores usam moedas sorteadas para pagar despesas e manter suas propriedades em andamento.
   - Possíveis premios incluem o Prêmio do Global e o role de "empréstimo" ou "devedor".

7. **Etapa de Vencedor:**
   - O objetivo é ganhar dinheiro, mas o jogo também inclui premios baseados no patrimônio acumulado.
   - Se um jogador tiver muito dinheiro em contraposição aos demais jogadores, pode ser "empréstimo" ou "devedor".

**Conclusão:**
Para aprender a jogar Monopoly, é essencial compreender as regras básicas, a etapa de começo, como lidar com brushing e specialidades, e como manipular o estoque de moedas. Com time spent practicando e experimentando diferentes estratégias, um jogador pode melhorar suas habilidades e aumentar as chances de vencer.

In [ ]:
display(Markdown(chain.invoke("Explique um Voice-Enabled AI e qual a conclusão dos autores ?")))

<think>
Primeiro, preciso entender o que é um Voice-Enabled AI. Parece que ele é um tipo de inteligência artificial que está associado ao voz, ou seja, pode ser controlado com as palavras de um usuário. Isso inclui habilidades como falar respostas, assistir a videoaulas e realizar tarefas específicas.

Agora, leio o texto que o usuário forneceu. Ele parece estar falando sobre uma avaliação ou uma análise de um método para usar AI em data science, comumente chamado de DS-1000. Na seção onde ele menciona "propRIETARIOS" e "abordagens", estou acreditando que ele esteja falando sobre um estudo que comparou diferentes tipos de modelo de AI propRIETARIOS contra os Aberto-sourse.

Então, a pergunta é: "Explain um Voice-Enabled AI and what is the conclusion of the authors?" Isso sugere que ele quer a minha explicação sobre o que é o Voice-Enabled AI e quais as conclusões drawn dos autores do texto fornecido.

No entanto, o texto fornecido não menciona diretamente nothing sobre Voice-Enabled AI. Parece que ele esteja mais focado na avaliação de modelos de AI propRIETARIOS e suas abordagens em data science, como usar Chain-of-Thought Research ou Instruções de Passagem Rápida (Five-shot prompting) com exemplos relevantes.

Pois, pode-se inferir que a conclusão dos autores é sobre a performance dos modelos propRIETARIOS em tarefas específicas de data science, especialmente em comparação com os Aberto-sourse. Eles mentionam que os models propRIETARIOS performaram melhor, especialmente nos benchmarks como DS-1000, onde models propRIETARIOS superaram significativamente os Aberto-sourse.

Portanto, a conclusão dos autores, no contexto do texto fornecido, parece ser que os models propRIETARIOS são mais indicados para tarefas de data science devido à melhor performance em benchmarks relevantes, e que usar abordagens como Chain-of-Thought Research pode ajudar a melhorar ainda mais o desempenho dos models.

Finalmente, para responder à pergunta, preciso congregar essas informações: explica brevemente o Voice-Enabled AI e menciona as conclusões dos autores sobre os modelos propRIETARIOS em data science.
</think>

Um **Voice-Enabled AI** (IntelliAGente com habilidades de voz) é uma inteligência artificial que pode ser controlada e interagido com através de palavras. Este tipo de AI pode executar tarefas como falar respostas, processar speech-to-text, assistir a videoaulas e realizar diversas operações específicas, tornando-o útil em contextos como atendimento ao cliente, instruções de trabalho ou interação educationais.

A **conclusão dos autores** dos estudos mencionados parece ser que os modelos propRIETARIOS ( proprietary AI models) demonstraram uma performance superior em tarefas de data science, especialmente em benchmarks como DS-1000. Em particular, os models propRIETARIOS superaram significativamente os models Aberto-sourse ( open-source models), com destaque para a capacidade de Chain-of-Thought Research e abordagens como Instruções de Passagem Rápida (Five-shot prompting) com exemplos relevantes em melhorar o desempenho dos models. Assim, os autores sugerem que os models propRIETARIOS são mais indicados para tarefas de data science devido à melhor performance em benchmarks relevantes.

In [28]:
display(Markdown(chain.invoke("Como executar o DeepSeek com python local ?")))

<think>
Para executar o DeepSeek com Python local, siga os seguintes passos:

1. **Baixar e Configurar o Framework**:
   - Download o framework DeepSeek do Repositório oficial.
   - Conecte ao Git para atualizar as dependentes.

2. ** instalar As Dependências**:
   - Execute `git install .` no terminal para baixar e instalar todas as dependências necessárias.

3. **Configurar a Pasta de Trabalho**:
   - Definir a pasta raiz onde estão armazenados os arquivos do projeto.

4. **Treinar o Modelo**:
   - Utilize o script `train.py` para treinar o modelo DeepSeek com as configurações desejadas.

5. **Carregar e Executar o Modelo**:
   - Carregue o modelo treinado em um ambiente Python local.
   - Execute as instruções específicas para interagir com o modelo DeepSeek.

6. **Testar em Produzir Texto**:
   - Teste diferentes tarefas de geração de texto para avaliar a performance do modelo.

7. **Optimizar e Ajustar Parâmetros**:
   - Realize ajustes necessários nas configurações para otimização do desempenho.
- **Documentação e Suporte**:
   - Consulte as documentações disponíveis para resolver qualquer dúvida ou problema em torno do uso do DeepSeek.

By following these steps, you can successfully run DeepSeek in your local Python environment and start utilizing its powerful text generation capabilities.
</think>

Para executar o DeepSeek com Python local, siga os seguintes passos:

### 1. Baixar e Configurar o Framework
Primeiramente, você precisará obter o Repositório do DeepSeek no GitHub para que possa configurá-lo corretamente.

```bash
git clone https://github.com/yourusername/deepseek.git
cd deepseek
```

### 2. Instalar as Dependências
Antes deanything, atualize as dependências do framework:

```bash
git install .
```

### 3. Configurar a Pasta de Trabalho
Definir a pasta raiz onde estão armazenados os arquivos do projeto:

```python
BASEPATH = '/path/to/your/project'
```

### 4. Treinar o Modelo
Treine o modelo usando o script de treinamento fornecido:

```bash
python train.py --config config/training.cfg
```

### 5. Carregar e Executar o Modelo
Carregue o modelo treinado e execute as instruções desejadas para interagir com o DeepSeek.

```python
from deepseek import DeepSeek

# Caminho para o arquivo do modelo treinado
model_path = 'path/to/trained_model'

# Inicialize o cliente
client = DeepSeek(
    model_dir=model_path,
    base_path=BASEPATH,
)

# Execute uma instrução
response = client.run("Resolva: Digite um texto em português e eu responderei com um texto em inglês, "
                      "usando as minas para identificar excessos de conteúdo ou excessos de vocabulário.")
print(response)
```

### 6. Testar em ProduzIR Texto
Geralmente, após o treinamento, você pode testar a capacidade de geração de texto com instruções específicas.

```python
# Gera um texto com base em uma instrução
prompt = "Dado que a cidade tem 10.000 habitantes e 5 estabelecimentos de saneamento, "
         "qual é a expectativa de população por km²?"
 
response = client.generate_text(
    prompt=prompt,
    max_tokens=100,
)
print("Resposta:", response)
```

### 7. Optimize e Ajustar Parâmetros
Seu modelo pode precisar de ajustes específicos, como alterar hiperparâmetros de treinamento ou otimizar para uma determinada tarefa.

```python
# Exemplo de ajuste noprompto
response = client.run(f"Treine um modelo com learning rate 0.001 e batch size de 32.")
```

### Documentação e Suporte
Para mais informações sobre as funcionalidades do DeepSeek, consulte a documentação disponível na seção de documento do Repositório GitHub.

```bash
cd deepseek/docs
```

### Conclusão
Com estas passos, você está prontinho para usar o DeepSeek no seu local em Python. Basta executar os comandos e ajustar conforme necessário suas instruções.

# Excluir Coleção

In [ ]:
# Delete all collections in the db
vector_db.delete_collection()
print(f"Coleção '{collection_name}' deletada.")